In [22]:
path_org='../SentenceGeneration/Data/DebiasingCorpus/corpus(10-40)10k.txt'

In [23]:
path_sim='../Notebook/output_generated_sentences_100_low_ttr.txt'

In [24]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

nltk.download('punkt')  # Downloading necessary NLTK data

def calculate_total_tokens_and_ngrams(sentences):
    total_tokens = 0
    unique_tokens = set()

    for sentence in sentences:
        # Tokenizing the sentence
        tokens = word_tokenize(sentence)
        total_tokens += len(tokens)

        # Updating the set of unique tokens
        unique_tokens.update(tokens)

    total_unique_tokens = len(unique_tokens)

    return total_tokens/len(sentences), total_unique_tokens/len(sentences)





[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [25]:
same_sentences = []
with open(path_sim, 'r', encoding='utf-8') as file:
    for line in file:
        # Strip to remove any leading/trailing whitespace
        sentence = line.strip()
        # Only add non-empty sentences
        if sentence:
            same_sentences.append(sentence)


In [26]:
same_sentences

["As the trump administration's red line was crossed, with a white house spokesman issuing a rare rebuke of netanyahu, the israeli-palestinian conflict reaches a boiling point, with the us government's stance on the issue becoming increasingly divisive and contentious, highlighting the urgent need for a comprehensive and lasting resolution to the decades-long dispute.",
 "In order to stay ahead in the competitive job market, California's diverse workforce is undergoing comprehensive upskilling, with Governor Gavin Newsom and his adviser Lenny Mendonca at the forefront of this initiative, ensuring that the state's workforce is well-equipped to meet the evolving demands of the ever-changing job landscape.",
 'Moreover, there is a lack of recognition for the workshops that aim to empower women and the successful organization of small-scale local elections, which are essential for the development of the country.',
 "In light of Trump's recent summit with Kim, it is evident that his disrega

In [27]:
org_sentences = []
with open(path_org, 'r', encoding='utf-8') as file:
    for line in file:
        # Strip to remove any leading/trailing whitespace
        sentence = line.strip()
        # Only add non-empty sentences
        if sentence:
            org_sentences.append(sentence)

In [28]:
org_sentences=org_sentences[:100]
n=1

## Checking number of tokens

In [29]:
average_tokens, total_bigrams =  calculate_total_tokens_and_ngrams(same_sentences)
print(f"Average number of tokens per sentence: {average_tokens}")
print(f"Total number of grams in the tokens: {total_bigrams}")

Average number of tokens per sentence: 53.78
Total number of grams in the tokens: 17.86


In [30]:
average_tokens, total_bigrams =  calculate_total_tokens_and_ngrams(org_sentences)
print(f"Average number of tokens per sentence: {average_tokens}")
print(f"Total number of {n}-grams in the tokens: {total_bigrams}")

Average number of tokens per sentence: 29.25
Total number of 1-grams in the tokens: 12.22


In [31]:
len(org_sentences)

100

## Checking similarity

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


vectorizer = TfidfVectorizer()
# Calculate similarity for each pair of sentences
#for i in range(80,100):
for i in range(min(len(org_sentences), len(same_sentences))):
    # Vectorize the sentences
    tfidf_matrix = vectorizer.fit_transform([org_sentences[i], same_sentences[i]])
    # Calculate the cosine similarity
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

    print(f"Similarity of sentence {i+1}: {similarity[0][0]}")

Similarity of sentence 1: 0.37479733253367625
Similarity of sentence 2: 0.36305489402661806
Similarity of sentence 3: 0.5517048124346837
Similarity of sentence 4: 0.2957177203889212
Similarity of sentence 5: 0.2802979962606551
Similarity of sentence 6: 0.6651333427051621
Similarity of sentence 7: 0.4137149172702679
Similarity of sentence 8: 0.20493644727113666
Similarity of sentence 9: 0.48694169634324774
Similarity of sentence 10: 0.279854988433961
Similarity of sentence 11: 0.5433178109146724
Similarity of sentence 12: 0.2440056301802919
Similarity of sentence 13: 0.3452093535778776
Similarity of sentence 14: 0.3204314966452357
Similarity of sentence 15: 0.4575786131492186
Similarity of sentence 16: 0.3509744046810146
Similarity of sentence 17: 0.19266981025501337
Similarity of sentence 18: 0.5884650256522277
Similarity of sentence 19: 0.19373805658196744
Similarity of sentence 20: 0.3501783487796154
Similarity of sentence 21: 0.3488836473339292
Similarity of sentence 22: 0.158329787

## Checking text token ratio

In [ ]:
def calculate_ttr(sentences_list):
    def get_word_tokens(sentences):
        tokens = []
        for sentence in sentences:
            # Split each sentence into words
            words = sentence.split()
            tokens.extend(words)
        return tokens

    # Get word tokens from the given list
    tokens = get_word_tokens(sentences_list)

    # Calculate the number of types (unique words)
    types = len(set(tokens))

    # Calculate the number of tokens (total words)
    tokens_total = len(tokens)

    # Calculate the type-token ratio (TTR)
    ttr = types / tokens_total if tokens_total > 0 else 0

    return ttr

In [ ]:
ratio_list1 = calculate_ttr(org_sentences)
ratio_list2 = calculate_ttr(same_sentences)

print(f"Text-to-token ratio for list1: {ratio_list1:.2f}")
print(f"Text-to-token ratio for list2: {ratio_list2:.2f}")


In [ ]:
def calculate_hapax_legomenon_ratio(sentences_list):
    def get_word_tokens(sentences):
        tokens = []
        for sentence in sentences:
            # Split each sentence into words
            words = sentence.split()
            tokens.extend(words)
        return tokens

    # Get word tokens from the given list
    tokens = get_word_tokens(sentences_list)

    # Calculate the number of hapax legomena (words occurring only once)
    hapax_legomena = [word for word in set(tokens) if tokens.count(word) == 1]

    # Calculate the hapax legomenon ratio (HLR)
    hlr = len(hapax_legomena) / len(tokens) if len(tokens) > 0 else 0

    return hlr


In [ ]:
ratio_list1 = calculate_hapax_legomenon_ratio(org_sentences)
ratio_list2 = calculate_hapax_legomenon_ratio(same_sentences)

print(f"Text-to-token ratio for list1: {ratio_list1:.2f}")
print(f"Text-to-token ratio for list2: {ratio_list2:.2f}")


## Readability

In [ ]:
import textstat

def calculate_readability_scores(sentences):
    # Join the list of sentences into a single text
    text = ' '.join(sentences)

    # Calculate readability scores
    flesch_kincaid = textstat.flesch_kincaid_grade(text)
    gunning_fog = textstat.gunning_fog(text)

    return flesch_kincaid, gunning_fog


In [ ]:
flesch_kincaid_score, gunning_fog_score = calculate_readability_scores(same_sentences)

print(f"Flesch-Kincaid Grade Level: {flesch_kincaid_score}")
print(f"Gunning Fog Index: {gunning_fog_score}")


In [ ]:
flesch_kincaid_score, gunning_fog_score = calculate_readability_scores(org_sentences)

print(f"Flesch-Kincaid Grade Level: {flesch_kincaid_score}")
print(f"Gunning Fog Index: {gunning_fog_score}")